In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
import math
from sklearn.cluster import KMeans
%matplotlib inline
import sys

In [2]:
sys.path.append("..")
from framework import perspective_transform
from framework import visualize
from framework import edges_detection

In [3]:
# data_path = "/home/avk/study/1C/Diplom/data/NewData/samples/"
data_path = "/home/avk/study/1C/Diplom/PerspectiveDistortion/SampleData/"

In [4]:
# # for file_num, filename in enumerate(os.listdir(data_path)):
# #     full_file_path = os.path.join(data_path, filename)
# #     if file_num == 5:
# # #     if file_num == 2:
# #         break
# # file_num
# full_file_path = "/home/avk/study/1C/Diplom/PerspectiveDistortion/SampleData/IMG_20190317_170456.jpg"

In [5]:
for file_num, filename in enumerate(os.listdir(data_path)):
    full_file_path = os.path.join(data_path, filename)
#     if file_num == 3: # 6 # 7
    if file_num == 3:
        break
file_num

3

In [6]:
initial_image = cv2.imread(full_file_path)
visualize.visualize_image(initial_image)
initial_image.shape

(2448, 3264, 3)

# Уменьшение изображения

In [7]:
resized_image, resizing_ratio = edges_detection.reduce_image_size(initial_image)
visualize.visualize_image(resized_image)
resized_image.shape

(300, 400, 3)

# Размытие текста

In [8]:
# for times in range(1, 8):
#     smoothed_image = edges_detection.blur_image(resized_image, cv2.medianBlur, 5, times)
#     visualize.visualize_image(smoothed_image, str(times))

In [9]:
smoothed_image = edges_detection.blur_image(resized_image, cv2.medianBlur, 5, 3)
visualize.visualize_image(smoothed_image)

# Выделение границ

In [10]:
# edges = cv2.Canny(smoothed_image, 10, 50)
# visualize.visualize_image(edges)

In [11]:
edges = edges_detection.canny_edge_detector(smoothed_image)
visualize.visualize_image(edges)

# Построение линий Хафа

In [12]:
hough_lines = edges_detection.find_hough_lines(edges)
print(len(hough_lines[0]), len(hough_lines[1]))
assert(min(len(hough_lines[0]), len(hough_lines[1])) >= 2)

4 4


In [13]:
cdst = edges_detection.draw_lines_on_edges(edges, hough_lines)
visualize.visualize_image(cdst)
print(hough_lines)

[[array([[280.       ,   1.5009831]], dtype=float32), array([[16.       ,  1.5882496]], dtype=float32), array([[13.       ,  1.6057029]], dtype=float32), array([[276.       ,   1.5184364]], dtype=float32)], [array([[-20.       ,   3.1241393]], dtype=float32), array([[-368.       ,    3.0892327]], dtype=float32), array([[23.,  0.]], dtype=float32), array([[-371.      ,    3.106686]], dtype=float32)]]


# Поиск пересечений линий

In [14]:
intersections = edges_detection.segmented_intersections(hough_lines)
len(intersections)

16

In [15]:
for point in intersections:
    cv2.circle(cdst, point, 3, (0, 255, 0), -1)
visualize.visualize_image(cdst)

# Кластеризация точек пересечения

In [21]:
corners = KMeans(n_clusters=4).fit(np.array(intersections)).cluster_centers_

In [24]:
for corner in corners:
    corner = tuple(np.array(corner, dtype=int))
    cv2.circle(cdst, corner, 3, (255, 0, 0), -1)
visualize.visualize_image(cdst)

# Работа с изначальным (неуменьшенным) изображением

In [19]:
corners = np.array(corners * resizing_ratio, dtype=int)
corners

array([[ 195, 2260],
       [3027,  195],
       [ 175,  122],
       [3108, 2080]])

In [20]:
warped = perspective_transform.remove_perspective_distortion(initial_image, corners)
visualize.visualize_image(warped)
warped.shape, warped.shape[0] / warped.shape[1]

((2037, 3061, 3), 0.66546880104541)